In [ ]:
import itertools
import os
from pathlib import Path
from typing import Any

import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
# from tensorflow.keras.layers.experimental import preprocessing

# 📥Load and Transform Data

In [ ]:
BATCH_SIZE = 32
TARGET_SIZE = (224, 224)
INPUT_SHAPE = (TARGET_SIZE[0], TARGET_SIZE[1], 3)
RANDOM_SEED = 1313

dataset = "../../data"

# 📅Placing data into a Dataframe
The first column `filepaths` contains the file path location of each individual images. The second column `labels`, on the other hand, contains the class label of the corresponding image from the file path

In [ ]:
image_dir = Path(dataset)

# Get filepaths and labels
extensions = ["JPG", "jpg", "png", "PNG"]
images_for_extensions = [list(image_dir.glob(rf"**/*.{extension}")) for extension in extensions]
filepaths_with_duplicates = [image for images_for_extension in images_for_extensions for image in images_for_extension]
filepaths = list(set(filepaths_with_duplicates))


def get_label_for_path(path: Path) -> str:
    # data/<LABEL>/<fileName>
    path_without_filename = os.path.split(path)[0]
    label = os.path.split(path_without_filename)[1]
    return label


labels = [get_label_for_path(x) for x in filepaths]

filepaths = pd.Series(filepaths, name="Filepath").astype(str)
labels = pd.Series(labels, name="Label")

# Concatenate filepaths and labels
images_df = pd.concat([filepaths, labels], axis=1)

print(images_df)

In [ ]:
label_counts = images_df["Label"].value_counts()

fix, ax = plt.subplots(figsize=(20, 6))
sns.barplot(x=label_counts.index, y=label_counts.values, alpha=0.8, palette="rocket")
ax.set_title("Cantidad de imagenes de cada animal en el dataset", fontsize=16)
ax.set_xlabel("Animal", fontsize=14)
ax.set_ylabel("Cantidad", fontsize=14)
ax.tick_params(axis="x", labelrotation=45)
plt.show()

# 🔭Visualizing images from the dataset

In [ ]:
# Display 16 pictures of the dataset with their labels
rows = 4
columns = 4
random_indexes = np.random.randint(0, len(images_df), rows * columns)
fig, axes = plt.subplots(nrows=rows, ncols=columns, figsize=(10, 10), subplot_kw={"xticks": [], "yticks": []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(images_df["Filepath"][random_indexes[i]]))
    ax.set_title(images_df["Label"][random_indexes[i]])
plt.tight_layout()
plt.show()

# 📝Data Preprocessing
<p style="font-size:20px; font-family:verdana; line-height: 1.7em">The data will be split into three different categories: Training, Validation and Testing. The training data will be used to train the deep learning CNN model and its parameters will be fine tuned with the validation data. Finally, the performance of the data will be evaluated using the test data(data the model has not previously seen).</p>

In [ ]:
def train_validation_test_df() -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    train_df, validationtest_df = train_test_split(
        images_df,
        train_size=0.7,
        random_state=RANDOM_SEED,
        shuffle=True,
        stratify=images_df["Label"],
    )
    validation_df, test_df = train_test_split(
        validationtest_df,
        test_size=0.6,
        random_state=RANDOM_SEED,
        shuffle=True,
        stratify=validationtest_df["Label"],
    )
    return train_df, validation_df, test_df


def train_validation_test_generator() -> tuple[Any, Any, Any]:
    train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        # Uncomment one of `preprocessing_function` or `rescale`
        preprocessing_function=keras.applications.mobilenet_v2.preprocess_input,
        # rescale=1.0 / 255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode="nearest",
        # validation_split=0.2,
    )
    validation_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        # Uncomment one of `preprocessing_function` or `rescale`
        preprocessing_function=keras.applications.mobilenet_v2.preprocess_input,
        # rescale=1.0 / 255,
    )
    test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        # Uncomment one of `preprocessing_function` or `rescale`
        preprocessing_function=keras.applications.mobilenet_v2.preprocess_input,
        # rescale=1.0 / 255,
    )
    return train_generator, validation_generator, test_generator


train_df, validation_df, test_df = train_validation_test_df()
train_generator, validation_generator, test_generator = train_validation_test_generator()

In [ ]:
def train_validation_test_images() -> tuple[Any, Any, Any]:
    train_images = train_generator.flow_from_dataframe(
        dataframe=train_df,
        x_col="Filepath",
        y_col="Label",
        target_size=TARGET_SIZE,
        batch_size=BATCH_SIZE,
        color_mode="rgb",
        class_mode="categorical",
        shuffle=True,
        seed=RANDOM_SEED,
        # subset="training",
    )
    validation_images = train_generator.flow_from_dataframe(
        dataframe=validation_df,
        x_col="Filepath",
        y_col="Label",
        target_size=TARGET_SIZE,
        batch_size=BATCH_SIZE,
        color_mode="rgb",
        class_mode="categorical",
        shuffle=True,
        # shuffle=False,
        seed=RANDOM_SEED,
        # subset="validation",
    )
    test_images = test_generator.flow_from_dataframe(
        dataframe=test_df,
        x_col="Filepath",
        y_col="Label",
        target_size=TARGET_SIZE,
        batch_size=BATCH_SIZE,
        color_mode="rgb",
        class_mode="categorical",
        shuffle=False,
    )
    return train_images, validation_images, test_images


# Split the data into three categories.
train_images, validation_images, test_images = train_validation_test_images()

In [ ]:
classes = list(train_images.class_indices.keys())
num_classes = len(classes)
assert num_classes == 23
print(classes)

# 🤹Training the model
<p style="font-size:20px; font-family:verdana; line-height: 1.7em">The model images will be subjected to a pre-trained CNN model called MobileNetV2. Three callbacks will be utilized to monitor the training. These are: Model Checkpoint, Early Stopping, Tensorboard callback. The summary of the model hyperparameter is shown as follows:</p>

<p style="font-size:20px">
  <strong>Batch size</strong>: 32<br>
  <strong>Epochs</strong>: 100<br>
  <strong>Input Shape</strong>: (224, 224, 3)<br>
  <strong>Output layer</strong>: 23
</p>

In [ ]:
# Load the pretained model
pretrained_model = keras.applications.MobileNetV2(
    input_shape=INPUT_SHAPE,
    include_top=False,
    weights="imagenet",
    # pooling="avg",
    # pooling="max",
    pooling=None,
)

pretrained_model.trainable = False

In [ ]:
def use_mobilenet_pretrained_model() -> Model:
    model = keras.Sequential()
    model.add(pretrained_model)
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(128, activation="relu"))
    model.add(keras.layers.Dropout(0.45))
    model.add(keras.layers.Dense(256, activation="relu"))
    model.add(keras.layers.Dropout(0.45))
    model.add(keras.layers.Dense(num_classes, activation="softmax"))
    return model


def use_custom_model() -> Model:
    model = keras.Sequential(
        [
            keras.layers.Input(shape=INPUT_SHAPE),
            keras.layers.Conv2D(32, (3, 3), activation="relu"),
            keras.layers.MaxPooling2D((2, 2)),
            keras.layers.Conv2D(64, (3, 3), activation="relu"),
            keras.layers.MaxPooling2D((2, 2)),
            keras.layers.Conv2D(128, (3, 3), activation="relu"),
            keras.layers.MaxPooling2D((2, 2)),
            keras.layers.Flatten(),
            keras.layers.Dense(128, activation="relu"),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(num_classes, activation="softmax"),
        ]
    )
    return model


model = use_mobilenet_pretrained_model()

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(), loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

# 🚄Training the model

In [ ]:
epochs = 100
# epochs = 10

history = model.fit(
    train_images,
    steps_per_epoch=len(train_images),
    validation_data=validation_images,
    validation_steps=len(validation_images),
    epochs=epochs,
    callbacks=[
        # early_stopping,
        # create_tensorboard_callback("training_logs", "animal_classification"),
        # checkpoint_callback,
    ],
)

In [ ]:
model.save_weights("animals_eff.weights.h5")

# ✔️Model Evaluation
<p style="font-size:20px; font-family:verdana; line-height: 1.7em">The test dataset will be used to evaluate the performance of the model.One of the metrics that will be tested would be accuracy which measures the fraction of predictions the model got right. Other metrics are as follows:   </p>

<h3>Precision(P):</h3> 
<p style="font-size:20px; font-family:verdana; line-height: 1.7em">The fraction of true positives (TP, correct predictions) from the total amount of relevant results, i.e., the sum of TP and false positives (FP). For multi-class classification problems, P is averaged among the classes. The following is the formula for precision.</p>

<h4>
  <center>
    <span style="font-size: 1.5em">
      $P = \frac{TP}{TP+FP}$
    </span>
  </center>
</h4>


<h3>Recall(R): </h3> 
<p style="font-size:20px; font-family:verdana; line-height: 1.7em">The fraction of TP from the total amount of TP and false negatives (FN). For multi-class classification problems, R gets averaged among all the classes. The following is the formula for recall.</p>

<h4>
  <center>
    <span style="font-size: 1.5em">
      $R = \frac{TP}{TP+FN}$
    </span>
  </center>
</h4>


<h3>F1 score(F1): </h3>

<p style="font-size:20px; font-family:verdana; line-height: 1.7em">The harmonic mean of precision and recall. For multi-class classification problems, F1 gets averaged among all the classes. The following is the formula for F1 score.</p>

<h4>
  <center>
    <span style="font-size: 1.5em">
      $F1 = 2 \times \frac{TP \times FP}{TP + FP}$
    </span>
  </center>
</h4>





In [ ]:
results = model.evaluate(test_images, verbose=0)

print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

# 📉Visualizing loss curves

In [ ]:
print(history.history.keys())

accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]

loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(len(accuracy))
plt.plot(epochs, accuracy, "b", label="Training accuracy")
plt.plot(epochs, val_accuracy, "r", label="Validation accuracy")

plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "b", label="Training loss")
plt.plot(epochs, val_loss, "r", label="Validation loss")

plt.title("Training and validation loss")
plt.legend()
plt.show()

# 🔮Making predictions on the Test Data

In [ ]:
# Predict the label of the test_images
pred = model.predict(test_images)
pred = np.argmax(pred, axis=1)

# Map the label
labels = train_images.class_indices
labels = dict((v, k) for k, v in labels.items())
pred = [labels[k] for k in pred]

# Display the result
print(f"The first 5 predictions: {pred[:5]}")

In [ ]:
# Display 25 random pictures from the dataset with their labels
random_indexes = np.random.randint(0, len(test_df) - 1, 15)
fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(25, 15), subplot_kw={"xticks": [], "yticks": []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(test_df.Filepath.iloc[random_indexes[i]]))
    if test_df.Label.iloc[random_indexes[i]] == pred[random_indexes[i]]:
        color = "green"
    else:
        color = "red"
    ax.set_title(f"True: {test_df.Label.iloc[random_indexes[i]]}\nPredicted: {pred[random_indexes[i]]}", color=color)
plt.show()
plt.tight_layout()

# 📊Plotting the Classification Reports and Confusion Matrix

<p style="font-size:20px; font-family:verdana; line-height: 1.7em"><b>Confusion matrix</b> and <b>classification report</b> are two important tools used for evaluating the performance of an image classification model.</p>

<p style="font-size:20px; font-family:verdana; line-height: 1.7em">A <b>confusion matrix</b> is a table that summarizes the number of correct and incorrect predictions made by a classification model on a set of test data. It is usually represented as a square matrix with rows and columns representing the predicted and true class labels, respectively. The entries of the matrix indicate the number of test samples that belong to a certain class, and how many of those were classified correctly or incorrectly by the model. A confusion matrix can provide a detailed breakdown of the performance of the model, including measures such as accuracy, precision, recall, and F1-score for each class. It can be used to identify specific areas where the model is making errors, and to diagnose problems with the model's predictions.</p>

<p style="font-size:20px; font-family:verdana; line-height: 1.7em">A <b>classification report</b> is a summary of the key performance metrics for a classification model, including precision, recall, and F1-score, as well as the overall accuracy of the model. It provides a concise overview of the model's performance, typically broken down by class, and can be used to quickly assess the strengths and weaknesses of the model. The report is often presented as a table, with each row representing a class and columns showing various performance metrics. The report may also include other metrics such as support (the number of test samples belonging to a particular class), and the macro- and micro-averages of the performance metrics across all classes.</p>

<p style="font-size:20px; font-family:verdana; line-height: 1.7em">In image classification, both confusion matrix and classification report are important tools for evaluating the performance of the model, identifying areas for improvement, and making decisions about how to adjust the model's architecture or training parameters.</p>

In [ ]:
y_test = list(test_df.Label)
print(classification_report(y_test, pred))

In [ ]:
report = classification_report(y_test, pred, output_dict=True)
df = pd.DataFrame(report).transpose()
df

In [ ]:
def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(20, 14), text_size=10, norm=False, savefig=False):
    """Makes a labelled confusion matrix comparing predictions and ground truth labels.

      If classes is passed, confusion matrix will be labelled, if not, integer class values
    will be used.

    Args:
      y_true: Array of truth labels (must be same shape as y_pred).
      y_pred: Array of predicted labels (must be same shape as y_true).
      classes: Array of class labels (e.g. string form). If `None`, integer labels are used.
      figsize: Size of output figure (default=(10, 10)).
      text_size: Size of output figure text (default=15).
      norm: normalize values or not (default=False).
      savefig: save confusion matrix to file (default=False).

    Returns:
      A labelled confusion matrix plot comparing y_true and y_pred.

    Example usage:
      make_confusion_matrix(y_true=test_labels, # ground truth test labels
                            y_pred=y_preds, # predicted labels
                            classes=class_names, # array of class label names
                            figsize=(15, 15),
                            text_size=10)
    """
    # Create the confustion matrix
    cm = confusion_matrix(y_true, y_pred)
    cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]  # normalize it
    n_classes = cm.shape[0]  # find the number of classes we're dealing with

    # Plot the figure and make it pretty
    fig, ax = plt.subplots(figsize=figsize)
    cax = ax.matshow(cm, cmap=plt.cm.Blues)  # colors will represent how 'correct' a class is, darker == better
    fig.colorbar(cax)

    # Are there a list of classes?
    if classes:
        labels = classes
    else:
        labels = np.arange(cm.shape[0])

    # Label the axes
    ax.set(
        title="Confusion Matrix",
        xlabel="Predicted label",
        ylabel="True label",
        xticks=np.arange(n_classes),  # create enough axis slots for each class
        yticks=np.arange(n_classes),
        xticklabels=labels,  # axes will labeled with class names (if they exist) or ints
        yticklabels=labels,
    )

    # Make x-axis labels appear on bottom
    ax.xaxis.set_label_position("bottom")
    ax.xaxis.tick_bottom()
    ### Added: Rotate xticks for readability & increase font size (required due to such a large confusion matrix)
    plt.xticks(rotation=90, fontsize=text_size)
    plt.yticks(fontsize=text_size)

    # Set the threshold for different colors
    threshold = (cm.max() + cm.min()) / 2.0

    # Plot the text on each cell
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if norm:
            plt.text(
                j,
                i,
                f"{cm[i, j]} ({cm_norm[i, j] * 100:.1f}%)",
                horizontalalignment="center",
                color="white" if cm[i, j] > threshold else "black",
                size=text_size,
            )
        else:
            plt.text(
                j,
                i,
                f"{cm[i, j]}",
                horizontalalignment="center",
                color="white" if cm[i, j] > threshold else "black",
                size=text_size,
            )

    # Save the figure to the current working directory
    if savefig:
        fig.savefig("confusion_matrix.png")


In [ ]:
make_confusion_matrix(y_test, pred, list(labels.values()))